In [1]:
# global var to keep track of test results we have
ERRORS = []
PASSES = []

### Write Permission to Home Directory Test

In [2]:
import subprocess
import os
def shelltest(CMD):
    global ERRORS
    try:
        result = subprocess.check_output(CMD, shell=True, stderr=subprocess.STDOUT)
        return 0, result.decode('utf-8')
    except subprocess.CalledProcessError as e:
        return e.returncode, e.output.decode('utf-8')
    except Exception as e:  # new add: handle exceptions 
        return -1, str(e)

# Test to check write permissions to home directory
TEST = "WRITE PERMISSION TO HOME DIRECTORY"
CMD = f"touch {os.path.expanduser('~')}/test_write_permissions.tmp && echo 'Write Permission: Yes' && rm {os.path.expanduser('~')}/test_write_permissions.tmp || echo 'Write Permission: No'"

# Execute Test
e, output = shelltest(CMD)
if e == 0:
    PASSES.append("Write Permission to Home Directory test")
else:
    ERRORS.append(output)

### UID/GID test

In [3]:
import os

# Compare UID and GID from Operating System and notebook, make sure they match
real_uid = os.getuid()
notebook_uid = !id -u

real_gid = os.getgid()
notebook_gid = !id -g

if real_uid != int(notebook_uid[0]) or real_gid != int(notebook_gid[0]):
    ERRORS.append("UID/GID test ERROR: UID and GID from Operating System and notebook do not match")
else:
    PASSES.append("UID/GID test")

### Environmental Variables test

In [4]:
#Load environment variables from system into python

OPE_UID = os.environ['OPE_UID']
OPE_GID = os.environ['OPE_GID']
OPE_GROUP = os.environ['OPE_GROUP']
NB_GROUP = os.environ['NB_GROUP'] 
NB_USER = os.environ['NB_USER']
XDG_CACHE_HOME = f"/home/{NB_USER}/.cache/"
#Verify environment variables are correct
notebook_uid = !id -u
notebook_gid = !id -g

err = []
if int(notebook_uid[0]) != int(OPE_UID) or int(notebook_gid[0]) != int(OPE_GID):
    err.append("Notebook UID or GID does not meet expected values.")
    
if OPE_GROUP != 'root' or NB_GROUP != 'root':
    err.append("OPE_GROUP Or NB_GROUP does not match 'root'.")
    
if NB_USER != 'jovyan':
    err.append("NB_USER does not match 'jovyan'.")
    
if XDG_CACHE_HOME != '/home/jovyan/.cache/':
    err.append("XDD_CACHE_HOME does not match expected path.")
if len(err) != 0:
    PASSES.append("Environmental Variables test")
else:
    s = '; '.join(err)
    ERRORS.append("Environmental Variables test ERROR: " + s)

KeyError: 'OPE_UID'

### ASLR test

In [11]:
# run gdb 100 times on date binary stopping at first instruction and grep ld. If ld is found at same address (following 
# command has a value of 1 then ASLR is properly disabled.
# We want to disable ASLR so that students can use GDB properly in their environments.
e, count = shelltest("echo $(for ((i=0;i<100; i++)); do gdb -ex starti -ex quit -q --batch /usr/bin/date 2>/dev/null | grep ld; done | tee out | uniq | wc -l)")
if count == 1: 
    PASSES.append("ASLR test")
else:
    e2, output2 = shelltest("echo $(cat /proc/sys/kernel/randomize_va_space)")
    ERRORS.append(f"ASLR test ERROR: ASLR status is enabled with status {output2} (1 is partial and 2 is full)")

                  

### Network Test

In [10]:
# Curl test to check internet connectivity
e, output = shelltest("curl google.com")
if e == 0:
    PASSES.append("Network test")
else:
    ERRORS.append("Network test ERROR: " + output)

### Pip-Conda Test

In [9]:
TEST = "PIP PACKAGE INSTALLATION"
CMD = "pip install --user pytest"
e, output = shelltest(CMD)

if e == 0:
    PASSES.append("Pip-Conda test")
else:
    ERRORS.append("Pip-Conda test ERROR: " + output)
print(ERRORS)

[]


### Test if Git and SSH configuration files are permanent and working

In [16]:
e, output = shelltest("readlink -f ~/.gitconfig")

if e == 0:
    PASSES.append("Git config test")
else:
    ERRORS.append("Git config test ERROR" + output)

In [15]:
e, output = shelltest("readlink -f /etc/ssh/ssh_config")

if e == 0:
    PASSES.append("ssh config test")
else:
    ERRORS.append("ssh config test ERROR" + output)

### Test if conda directories are read/writable

In [6]:
def check_permissions(dir_path):
    """Check if a directory is readable and writable."""
    global ERRORS
    try:
        readable = os.access(dir_path, os.R_OK)
        writable = os.access(dir_path, os.W_OK)
        return readable, writable
    except Exception as e:
        ERRORS.append("Conda directory r/w test" + f"checking permissions for {dir_path}.")
        return False, False

# Identify the conda directories
conda_base_dir = os.path.abspath(os.path.join(os.path.dirname(os.sys.executable), ".."))
conda_env_dir = os.environ.get('CONDA_PREFIX', '')

unaccessible_dirs = 0

for dir_name, dir_path in [('Conda Base Directory', conda_base_dir), ('Conda Environment Directory', conda_env_dir)]:
    readable, writable = check_permissions(dir_path)

    if not (readable and writable):
        unaccessible_dirs += 1
if unaccessible_dirs == 0:
    PASSES.append("Conda directory r/w test")

In [21]:
if len(ERRORS) > 0:
    print("FAILED TESTS:")
    print('-' + '\n-'.join(ERRORS))
    print('\n')
    print("PASSED TESTS:")
    print('-' + '\n-'.join(PASSES))
else:
    print("ALL TESTS PASS")
    print('-' + '\n-'.join(PASSES))

FAILED TESTS:
-ASLR test ERROR: ASLR status is enabled with status cat: /proc/sys/kernel/randomize_va_space: No such file or directory

 (1 is partial and 2 is full)


PASSED TESTS:
-Write Permission to Home Directory test
-UID/GID test
-Conda directory r/w test
-Git config test
-Pip-Conda test
-Network test
-ssh config test
-Git config test
